<a href="https://colab.research.google.com/github/jacobeturpin/CRX14/blob/master/CRX14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting Thorax Diseases Using the ChestX-Ray14 Dataset and Convolutional Techniques

Dataset provided by the National Institute of Health at: https://nihcc.app.box.com/v/ChestXray-NIHCC

In [0]:
%tensorflow_version 2.x
!pip install keras --upgrade --quiet
!pip install efficientnet --quiet

In [0]:
import gzip
import os
from urllib.request import urlretrieve

import pandas as pd

import keras
from keras.applications import DenseNet121, ResNet50V2
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Flatten
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator

import efficientnet.keras as efn

In [0]:
DATA_PATH = './samples/'
BATCH_SIZE = 16

## Data Loading

In [0]:
def batch_download(first_n=None):

  # URLs for the zip files
  links = [
    'https://nihcc.box.com/shared/static/vfk49d74nhbxq3nqjg0900w5nvkorp5c.gz',
    'https://nihcc.box.com/shared/static/i28rlmbvmfjbl8p2n3ril0pptcmcu9d1.gz',
    'https://nihcc.box.com/shared/static/f1t00wrtdk94satdfb9olcolqx20z2jp.gz',
    'https://nihcc.box.com/shared/static/0aowwzs5lhjrceb3qp67ahp0rd1l1etg.gz',
    'https://nihcc.box.com/shared/static/v5e3goj22zr6h8tzualxfsqlqaygfbsn.gz',
    'https://nihcc.box.com/shared/static/asi7ikud9jwnkrnkj99jnpfkjdes7l6l.gz',
    'https://nihcc.box.com/shared/static/jn1b4mw4n6lnh74ovmcjb8y48h8xj07n.gz',
    'https://nihcc.box.com/shared/static/tvpxmn7qyrgl0w8wfh9kqfjskv6nmm1j.gz',
    'https://nihcc.box.com/shared/static/upyy3ml7qdumlgk2rfcvlb9k6gvqq2pj.gz',
    'https://nihcc.box.com/shared/static/l6nilvfa9cg3s28tqv1qc1olm3gnz54p.gz',
    'https://nihcc.box.com/shared/static/hhq8fkdgvcari67vfhs7ppg2w6ni4jze.gz',
    'https://nihcc.box.com/shared/static/ioqwiy20ihqwyr8pf4c24eazhh281pbu.gz'
  ]

  first_n = first_n or len(links)

  if first_n > len(links):
    raise('Number of files requested exceeds amount available')

  for idx, link in enumerate(links[:first_n]):
      fn = 'images_%02d.tar.gz' % (idx+1)
      print('downloading', fn, '...')
      urlretrieve(link, fn)  # download the zip file
  
  print("Download complete. Please check the checksums")

In [0]:
#batch_download(1)

In [0]:
def extract_gzip(filename):
  f = gzip.open(filename, 'rb')
  file_content = f.read()
  # TODO: implement save decompressed
  f.close()

In [0]:
for fn in os.listdir():
  if '.tar.gz' in fn:
    extract_gzip(fn)

## Data Preparation

The ChestX-ray14 dataset is too large to fit entirely in memory when training; therefore, it's incrementally loaded via generator to reduce memory overhead. This is achieved using the Keras [Image Proprocessing](https://keras.io/preprocessing/image/) submodule.

In [0]:
# TODO: implement
train_df = pd.DataFrame()
test_df = pd.DataFrame()

In [0]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.25
)
test_datagen = ImageDataGenerator(rescale=1./255)

In [0]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=DATA_PATH,
    x_col='file',
    y_col=[],  # TODO: add classes
    subset='training',
    batch_size=BATCH_SIZE,
    shuffle=True,
    class_mode='categorical',
    classes=[],  # TODO: implement
    target_size=(224, 224)
)

In [0]:
valid_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=DATA_PATH,
    x_col='file',
    y_col=[],  # TODO: add classes
    subset='validation',
    batch_size=BATCH_SIZE,
    shuffle=True,
    class_mode='categorical',
    classes=[],  # TODO: implement
    target_size=(224, 224)
)

In [0]:
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=DATA_PATH,
    x_col='file',
    y_col=[],  # TODO: add classes
    batch_size=BATCH_SIZE,
    shuffle=False,
    class_mode='categorical',
    classes=[],  # TODO: implement
    target_size=(224, 224)
)

## Modeling

Three models will be implemented and their results compared:

1.   ResNet
2.   DenseNet
3.   EfficientNet


### ResNet

A pre-built ResNet model from the Keras library is used. Documentation on the model can be found [here](https://keras.io/applications/). Pre-trained weights from the ImageNet dataset are used.

In [10]:
resnet_base = ResNet50V2(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3),
    pooling='avg'
)
output = Dense(14, activation='sigmoid')(resnet_base.output)

resnet = Model(input=resnet_base.input, outputs=output)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
  if __name__ == '__main__':


In [11]:
resnet.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
____________________________________________________________________________________________

In [0]:
resnet.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [0]:
# TODO: implement fit

### DenseNet

In [17]:
densenet_base = DenseNet121(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3),
    pooling='avg'
)
output = Dense(14, activation='sigmoid')(densenet_base.output)

densenet = Model(input=densenet_base.input, outputs=output)

29089792/29084464 [==============================] - 0s 0us/step


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
  if __name__ == '__main__':


In [18]:
densenet.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 230, 230, 3)  0           input_6[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
____________________________________________________________________________________________

In [0]:
densenet.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

### EfficientNet

EfficientNet is a lightweight CNN architecture that is designed to require significantly less compute than other state of the art architectures on common transfer learning datasets.

Pre-built EfficientNet models built in Keras are used from the efficientnet library available on [GitHub](https://github.com/qubvel/efficientnet) and installable via PyPI.

In [23]:
efficientnet_base = efn.EfficientNetB0(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3),
    pooling='avg'
)
output = Dense(14, activation='sigmoid')(efficientnet_base.output)

efficientnet = Model(input=efficientnet_base.input, outputs=output)

16809984/16804768 [==============================] - 0s 0us/step


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
  if __name__ == '__main__':


In [24]:
efficientnet.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 112, 112, 32) 864         input_7[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 112, 112, 32) 128         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 112, 112, 32) 0           stem_bn[0][0]                    
____________________________________________________________________________________________

In [0]:
efficientnet.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)